In [1]:
!pip install langchain langchain_google_genai langchain_community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [2]:
!pip install pymupdf easyocr pillow -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_google_genai import ChatGoogleGenerativeAI
import fitz  # PyMuPDF
import easyocr
import torch
import io
from PIL import Image
import torch

In [4]:
from google.colab import userdata
GOOGLE_GENAI_KEY = userdata.get('GOOGLE_GENAI_KEY')

In [5]:
llm = ChatGoogleGenerativeAI(model="models/gemini-pro",google_api_key=GOOGLE_GENAI_KEY)

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()

# Initialize EasyOCR reader (English only, add other languages if needed)
reader = easyocr.Reader(['en'], gpu=use_gpu)
print(use_gpu)
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = []

    for page in doc:
        text = page.get_text("text").strip()

        # Extract text if available
        if text:
            full_text.append(text)

        # Extract text from images if present
        for img in page.get_images(full=True):
            base_image = doc.extract_image(img[0])
            image_bytes = base_image["image"]
            img_text = "\n".join(reader.readtext(image_bytes, detail=0))

            if img_text:
                full_text.append(img_text)

    return "\n".join(full_text)

# Example Usage:
doc_path = r"/content/2025021580.pdf"
extracted_text = extract_text_from_pdf(doc_path)

True


In [8]:
# Text splitting configuration
text_splitter = RecursiveCharacterTextSplitter(chunk_size=6000, chunk_overlap=200)
chunks = text_splitter.split_text(extracted_text)

# Convert chunks into LangChain Document format
docs = [Document(page_content=chunk) for chunk in chunks]

In [9]:
summary_prompt = """
[Legal Document Summarization]
You are an advanced AI legal assistant specializing in document analysis and legal summarization.
**Your goal is to generate a structured, concise, and legally precise summary.**

---

## 1: Identify the Type of Legal Document**
Analyze the document and determine its type. Clearly classify it under one of the following legal categories:
- **Judgment / Court Order**
- **First Information Report (FIR)**
- **Legal Notice / Demand Notice**
- **Contract / Agreement**
- **Petition / Writ**
- **Other (Specify the exact legal category)**

---

## 2: Generate a Highly Structured Summary**
Extract and present key details in a well-organized format:

### **Document Overview**
- **Category:** [Specify the document type]
- **Title / Case Name / Subject:**
- **Parties Involved:** (Petitioner, Respondent, Plaintiff, Defendant, Complainant, Accused, etc.)
- **Date & Reference Number:** (Case No., FIR No., Agreement Date, etc.)
- **Issuing Authority / Court / Legal Body:** (e.g., Delhi High Court, Police Station XYZ, etc.)
- **Jurisdiction:** (City, State, Country)

---

### **Key Legal Content & Analysis**
- **Nature of the Matter:** (Civil, Criminal, Commercial, Property Dispute, Employment, Family Law, etc.)
- **Main Legal Issues or Allegations:** (Summarize the core legal concerns in precise legal language)
- **Claims / Relief Sought / Demands:** (What is being requested? Compensation, compliance, legal penalties, etc.)
- **Key Arguments / Evidence Cited:** (Summarize critical legal points, contract clauses, evidence, etc.)
- **Orders, Decisions, or Actions Taken (if applicable):** (Court orders, directives, penalties imposed, deadlines set)
- **Current Status:** (Pending, Resolved, Under Investigation, Awaiting Response, etc.)
- **Critical Deadlines & Next Steps:** (Legal response deadlines, compliance dates, upcoming hearings, etc.)

---

## 3: Risk Assessment & Legal Advisory (If Applicable)**
For **Court Orders, FIRs, and Legal Notices**, assess potential risks and recommend the next course of action:

- **Legal Risk Level:** [Low / Medium / High]
- **Potential Consequences:** (Possible legal liabilities, penalties, court actions, enforcement risks)
- **Recommended Legal Actions:** (Steps to be taken immediately—e.g., hiring a lawyer, filing a response, negotiation strategies, etc.)

Input Text:
{}

Output:
Provide a well-structured summary of the input text.
"""

In [10]:
# Summarize each chunk and extract text content
summarized_chunks = [llm.invoke(summary_prompt.format(doc.page_content)).content for doc in docs]

# Summarize the summarized chunks
final_summary = llm.invoke(summary_prompt.format("\n".join(summarized_chunks))).content

print(final_summary)

---

## 1: Identify the Type of Legal Document**
- **Category:** Judgment / Court Order

---

## 2: Generate a Highly Structured Summary**

### **Document Overview**
- **Title / Case Name / Subject:** Directions Regarding Unauthorized Construction
- **Parties Involved:** Not specified in the provided text
- **Date & Reference Number:** Not mentioned in the provided text
- **Issuing Authority / Court / Legal Body:** Supreme Court of India
- **Jurisdiction:** India

---

### **Key Legal Content & Analysis**
- **Nature of the Matter:** Unauthorized construction and demolition
- **Main Legal Issues or Allegations:**
    - Unauthorized construction is a serious issue that poses risks to occupants and the environment.
    - Regularization schemes should be exceptional and justified to ensure compliance with building plans and regulations.
    - Officials should be held accountable for failing to prevent unauthorized construction.
- **Claims / Relief Sought / Demands:**
    - Strict complianc